# 1-dimensional MD - harmonic well (Exercise 24)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rc("xtick", labelsize=12)
plt.rc("ytick", labelsize=12)
plt.rc("axes", titlesize=16)
plt.rc("font", size=12)
from scipy.spatial.distance import pdist,squareform
from scipy.optimize import fmin
from matplotlib.colors import to_rgba
from scipy.integrate import quad

In [ ]:
class Spring():
    def __init__(self,k=1,x0=0):
        self.k = k
        self.x0 = x0
        
    def potential_energy(self, x):
        return 0.5 * self.k * (x - self.x0)**2
    
    def force(self, x):
        return YOUR CODE


In [ ]:
from week5 import SimulationSystem

In [ ]:
class MonteCarloSystem(SimulationSystem):
        
    def __init__(self, *args, transition_method='delta', sample_size=1000, delta=0.1, **kwargs):
        super().__init__(*args, **kwargs)
        assert transition_method in ['delta','uniform'], 'Unknown transition method'
        self.transition_method = transition_method
        self.delta = delta
    
    def direct_integration(self, property=None):
        if property is None:
            property = self.calc.potential_energy
        numerator_inner = lambda x: property(x) * np.exp(-self.calc.potential_energy(x) / self.kT)
        denominator_inner = lambda x: np.exp(-self.calc.potential_energy(x) / self.kT)
        numerator = quad(numerator_inner, self.xmin, self.xmax)[0] 
        denominator = quad(denominator_inner, self.xmin, self.xmax)[0] 
        return numerator / denominator
    
    def estimate_from_sample(self, property=None):
        if property is None:
            property = self.calc.potential_energy
        if self.sample is None:
            self.setup_sample()
        numerator = np.sum(property(self.sample))
        denominator = len(self.sample)
        return numerator / denominator
        
    def setup_sample(self):
        
        xs = []
        x = self.x
        for e in range(self.sample_size):
            xs.append(x)
            if self.transition_method == 'delta':
                x_new = x + self.delta*np.random.randn()
            else:
                x_new = self.xmin + (self.xmax-self.xmin)*np.random.rand()
            de = self.calc.potential_energy(x_new) - self.calc.potential_energy(x)
            if np.random.rand() < np.exp(-de/self.kT):
                x = x_new
        self.sample = np.array(xs)
        
        print('Sample size:',len(self.sample))
        
    def plot(self,ax,xwidth=0.1):
        super().plot(ax,xwidth)
        
        average_V_exact = self.direct_integration()
        average_V_sampl = self.estimate_from_sample()
        ax.set_title(f'Ve={average_V_exact:.3f} Vmet-MC={average_V_sampl:.3f}')
        ax.text(-1,1,f'kT={self.kT}')

In [ ]:
spring = Spring()

fig, axes = plt.subplots(1, 4, figsize=(16,4))
system = MonteCarloSystem(spring, sample_size=1000, transition_method='uniform')
kTs = [0.05, 0.15, 0.25, 0.35]
for ax,kT in zip(axes,kTs):
    system.kT = kT
    system.plot(ax)
    ax.set_xlabel(r'x')
    ax.set_ylabel(r'V')
    ax.set_ylim([0, 1.5])

fig.tight_layout()
#fig.savefig('exercise_24_fig1.png')

In [ ]:
def velocity_verlet_1d(system, N=100):
    rs = []
    for _ in range(N):
        dt = 0.01
        r = system.get_position()
        v = system.get_velocity()
        a_t = system.force()
        r += YOUR CODE
        system.set_position(r)
        
        YOUR CODE
        
        system.set_velocity(v)
        rs.append(r)
        
    return rs

In [ ]:
class MolDynSystem(SimulationSystem):
    def __init__(self, *args, thermostat=None, verlet_steps=50, **kwargs):
        super().__init__(*args, **kwargs)
        self.v = 0
        self.thermostat = thermostat
        self.verlet_steps = verlet_steps
        
    def force(self):
        return self.calc.force(self.x)
        
    def get_velocity(self):
        return self.v
    
    def set_velocity(self,v):
        self.v = v
        
    def setup_sample(self):
        if self.sample is None:
            self.sample = []
        for _ in range(self.sample_size):
            r = velocity_verlet_1d(self,N=self.verlet_steps)
            self.sample.append(r[-1])
            if self.thermostat is not None:
                self.thermostat(self)
           
        
    def plot(self,ax,xwidth=0.1):
        super().plot(ax,xwidth)
        
        vpotave = np.mean([self.calc.potential_energy(x) for x in self.sample])
        ax.set_title(f'<Vpot>={vpotave:.3f}')

In [ ]:
system = MolDynSystem(spring, x=1, verlet_steps=1)

fig, axes = plt.subplots(1, 2, figsize=(8,4))

ax = axes[1]
system.plot(ax)
ax.set_xlabel(r'x')
ax.set_ylabel(r'V')
ax.set_ylim([0, 1.5])

ax = axes[0]
ax.plot(system.sample)

fig.tight_layout()

In [ ]:
def nvt_thermostat(md):
     YOUR CODE


In [ ]:
system = MolDynSystem(spring, thermostat=nvt_thermostat, kT=0.15)

fig, axes = plt.subplots(1, 2, figsize=(8,4))

ax = axes[1]
system.plot(ax)
ax.set_xlabel(r'x')
ax.set_ylabel(r'V')
ax.set_ylim([0, 1.5])

ax = axes[0]
ax.plot(system.sample)

fig.tight_layout()

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(16,4))
system = MolDynSystem(spring, thermostat=nvt_thermostat)
kTs = [0.05, 0.15, 0.25, 0.35]
for ax,kT in zip(axes,kTs):
    system.kT = kT
    system.plot(ax)
    ax.set_xlabel(r'x')
    ax.set_ylabel(r'V')
    ax.set_ylim([0, 1.5])

fig.tight_layout()